# About: Amazon S3からのリストア

---

Moodle構築環境のデータ、設定ファイルなどを Amazon S3 からリストアします。

## 概要

Amazon S3に作成したMoodle環境のバックアップからのリストアを行います。

### 前提条件


この Notebook を実行するには事前に以下のものを準備する必要があります。

* バックアップの保存先であるAmazon S3にアクセルするためのクレデンシャル
* リストア先となるVCノード/EC2インスタンス/Azure仮想マシン

リストア先となる環境は「011-VCノードの作成」、「012-EC2インスタンスの作成」、「013-Azure仮想マシンの作成」のいずれかのNotebookで作成することができます。

また、リストアの操作により既存の環境を損ねることを避けるため、リストア先となるディレクトリ `/srv/moodle` が存在していないことを前提としています。

## パラメータ設定

### リストア先のMoodle環境を指定する

リストア先となるノードをAnsibleのグループ名で指定してください。

既存のグループ名を確認するために`group_vars`にあるディレクトリの一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleのグループ名を次のセルに指定してください。

In [ ]:
# (例)
# target_group = 'Moodle'

target_group = 

#### チェック

指定された `target_group` の値が適切なものかチェックします。

`target_group` に対応する設定ファイルが存在していることを確認します。

In [ ]:
from pathlib import Path

target_dir = Path('group_vars') / target_group
if not (target_dir.is_dir() and len(list(target_dir.glob("*"))) > 0):
    raise RuntimeError(f"ERROR: not exists {target_group}")

リストア対象のノードにアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

リストア先となるディレクトリ `/srv/moodle` が存在していないことを確認します。

In [ ]:
!ansible {target_group} -a 'test ! -e /srv/moodle'

### AWSの認証情報

Amazon S3を利用するための認証情報を指定します。

Amazonのアクセスキーを指定してください。

In [ ]:
from getpass import getpass
aws_access_key = getpass()

Amazonのシークレットキーを指定してください。

In [ ]:
aws_secret_key = getpass()

### バックアップの保存先を指定する

バックアップの保存先である Amazon S3 のバケット名、パスを指定します。

Amazon S3 のバケット名を指定してください。

In [ ]:
# (例)
# s3_bucket = 's3-bucket-name'

s3_bucket = 

リストアを行うバックアップファイルを選択するために、バックアップファイルの一覧を確認します。

In [ ]:
!ansible {target_group} -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 ls --recursive s3://{s3_bucket}/'

リストアを行うバックアップファイルを格納しているディレクトリを指定してください。

In [ ]:
# (例)
# backup_dir = 'moodle-simple/Moodle/2020-XX-XXTXX:XX:XX.XXXXXX'

backup_dir = 

指定したバケット名、ディレクトリにバックアップファイルがあることを確認します。次のセルを実行してMoodle環境のバックアップファイル `db.sql.gz`, `moodle.tar.gz` が存在していることを確認してください。

In [ ]:
!ansible {target_group} -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 ls s3://{s3_bucket}/{backup_dir}/'

### MoodleのURL

リストア環境でMoodleのURLを変更する場合は、次のセルに新しいURLを指定してください。

> バックアップに記録されているURLをそのまま使用する場合は、この節をスキップしてください。

In [ ]:
# (例)
# moodle_url = 'http://moodle.example.org'
# moodle_url = 'http://172.30.2.100'
# moodle_url = 'https://moodle.example.org'

moodle_url = 

指定された値の簡単なチェックを行います。

In [ ]:
if 'moodle_url' in vars():
    %run scripts/utils.py
    check_parameters('moodle_url', nb_vars=locals())

## リストア

Moodle環境のリストアを行います。

### ディレクトリの作成

リストア先となるディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle state=directory owner={{{{ansible_ssh_user}}}}'

デフォルトのコンテナログの出力先ディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a 'path=/var/log/httpd state=directory'
!ansible {target_group} -b -m file -a 'path=/var/log/mysql owner=999 group=adm state=directory'

### コンテナ構成、Moodleデータなどのリストア

設定ファイルなどのバックアップファイル `moodle.tar.gz` をリストアします。

リストア対象のバックアップファイルを確認します。

In [ ]:
s3_moodle_backup = f's3://{s3_bucket}/{backup_dir}/moodle.tar.gz'
print(s3_moodle_backup)

リストアを行います。

In [ ]:
!ansible {target_group} -m shell -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 cp {s3_moodle_backup} - \
    | bash -c "sudo tar xzpf - -C /srv/moodle"'

リストアされたことを確認します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle tree -L 3 -F'

### DBデータ

DBデータを復元するためのSQLファイルを取得します。

リストア対象のバックアップファイルを確認します。

In [ ]:
s3_db_backup = f's3://{s3_bucket}/{backup_dir}/db.sql.gz'
print(s3_db_backup)

リストア先となるディレクトリを作成します。

In [ ]:
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle/db/data state=directory owner={{{{ansible_ssh_user}}}}'
!ansible {target_group} -b -m file -a \
    'path=/srv/moodle/db/sql state=directory owner={{{{ansible_ssh_user}}}}'

DBデータをリストアするためのSQLファイルファイルを配置します。配置したSQLファイルはDBコンテナ起動時に自動的に実行されます。

In [ ]:
!ansible {target_group} -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 cp {s3_db_backup} /srv/moodle/db/sql/backup.sql.gz'

### コンテナイメージ

Dockerコンテナイメージのリストアを行います。

> コンテナイメージのバックアップを作成していない、あるいはレポジトリのコンテナイメージを利用する場合はこの節を実行せずにスキップしてください。

コンテナイメージをリストアする前の状態を確認しておきます。コンテナイメージの一覧を表示します。

In [ ]:
!ansible {target_group} -a 'docker images'

リストア対象のバックアップファイルを確認します。

In [ ]:
s3_cnt_img = f's3://{s3_bucket}/{backup_dir}/container-image.tar.gz'
print(s3_cnt_img)

コンテナイメージをリストアします。

> バックアップファイルが存在していない場合はエラーになります。

In [ ]:
!ansible {target_group} -m shell -a \
    'env AWS_ACCESS_KEY_ID={aws_access_key} AWS_SECRET_ACCESS_KEY="{aws_secret_key}" \
    aws s3 cp {s3_cnt_img} - | gzip -cd | docker load'

リストアを行った後の状態を確認します。コンテナイメージの一覧を表示します。

In [ ]:
!ansible {target_group} -a 'docker images'

### group_vars

バックアップされた`group_vars`を、このJupyter Notebook環境にリストアします。

リストアにより意図せず既存のパラメータを変更してしまうことへの対応として、現在のgroup_varsを複製します。

In [ ]:
!cp -a group_vars/{target_group}  group_vars/.{target_group}.$(date +"%Y%m%d%H%M%S").bak

group_varsのリストアを行います。

In [ ]:
!ansible {target_group} -Dv -m synchronize -a \
    'src=/srv/moodle/misc/group_vars/*/ dest=group_vars/{target_group} mode=pull'

[2.4 MoodleのURL](#MoodleのURL)で新たな`moodle_url`を指定した場合はその値で `group_vars` を更新します。また新たな`group_vars`の値で`docker-compose.yml`を生成し直します。

In [ ]:
if 'moodle_url' in vars():
    %run scripts/group.py

    update_group_vars(
        target_group,
        moodle_url=moodle_url,
    )
    !ansible {target_group} -Dv -m template \
        -a 'src=template/docker-compose.yml.j2 dest=/srv/moodle/docker-compose.yml backup=yes'

### crontab

crontabの設定を復元します。

現在の crontab の設定を確認します。

In [ ]:
try:
    !ansible {target_group} -a 'crontab -l'
except:
    pass

バックアップファイルからcrontabの復元を行います。

In [ ]:
!ansible {target_group} -a 'crontab /srv/moodle/misc/crontab'

復元後の設定を確認します。

In [ ]:
!ansible {target_group} -a 'crontab -l'

### logrotate

コンテナログのログローテーションの設定を復元します。

In [ ]:
for file in ["httpd", "mysql-server"]:
    !ansible {target_group} -b -m copy -a \
        'src=/srv/moodle/misc/logrotate.d/{file} dest=/etc/logrotate.d/ remote_src=yes'

### rsyslog

rsyslogの設定ファイルを復元します。ただしバックアップした環境で`051-ロギングドライバを変更する.ipynb`によりrsyslogのロギング設定を行った場合のみ実行します。

In [ ]:
%run scripts/group.py
try:
    logging_driver = load_group_var(target_group, "logging_driver")
except KeyError:
    logging_driver = None

if logging_driver == "syslog":
    !ansible {target_group} -b -m copy -a \
        'src=/srv/moodle/misc/rsyslog/rsyslog.conf dest=/etc/ remote_src=yes' || true
    !ansible {target_group} -b -m copy -a \
        'src=/srv/moodle/misc/rsyslog/rsyslog.d/moodle.conf dest=/etc/rsyslog.d/ remote_src=yes' || true

設定を反映させるためにrsyslogサービスの再起動を行います。

In [ ]:
if logging_driver == "syslog":
    !ansible {target_group} -b -m systemd -a \
        'name=rsyslog state=restarted'

rsyslogサービスの状態を確認します。

In [ ]:
!ansible {target_group} -b -a \
    'systemctl status rsyslog'

### fluentd

fluentdの設定ファイルを復元します。ただしバックアップした環境で`052-ロギングドライバを変更する.ipynb`によりfluentdのロギング設定を行った場合のみ実行します。

In [ ]:
if logging_driver == "fluentd":
    !ansible {target_group} -b -m copy -a \
        'src=/srv/moodle/misc/fluentd/fluent.conf dest=/etc/fluentd/ remote_src=yes' || true
    !ansible {target_group} -b -m copy -a \
        'src=/srv/moodle/misc/fluentd/config.d/moodle.conf dest=/etc/fluentd/config.d/ remote_src=yes' || true
    !ansible {target_group} -b -m copy -a \
        'src=/srv/moodle/misc/fluentd/config.d/ocs-output/forward.conf dest=/etc/fluentd/config.d/ocs-output/ remote_src=yes' || true

設定を反映させるためにfluentdサービスの再読み込みを行います。

In [ ]:
if logging_driver == "fluentd":
    !ansible {target_group} -b -m systemd -a \
        'name=fluentd state=reloaded'

fluentdサービスの状態を確認します。

In [ ]:
if logging_driver == "fluentd":
    !ansible {target_group} -b -a \
        'systemctl status fluentd'

### コンテナの起動

リストアした環境のコンテナを起動します。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose up -d'

コンテナの状態を確認します。`State`が`Up`となっていることを確認してください。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle \
    docker compose ps'

### メンテナンスモードの解除

メンテナンスモードに変更してからバックアップを作成した場合はメンテナンスモードの解除が必要となります。

In [ ]:
!ansible {target_group} -a 'chdir=/srv/moodle docker compose exec -T moodle \
    /usr/bin/php /var/www/html/admin/cli/maintenance.php --disable'

## Moodle を利用できることを確認

リストアしたMoodle環境にアクセスして利用できることを確認します。

次のセルを実行するとリストアしたMoodle環境にアクセスするためのリンクが表示されます。リンクをクリックしてMoodle環境にログインできることを確認してください。

In [ ]:
from IPython.core.display import HTML
%run scripts/group.py
moodle_url = load_group_var(target_group, "moodle_url")
HTML(u'<a href="{0}" target="_blank">{0}</a>'.format(moodle_url))